<a href="https://colab.research.google.com/github/wingated/cs474_labs_f2019/blob/master/DL_Lab4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cancer Detection

## Overview
Convolutional neural networks are used to solve many computer vision problems. One of the more modern problems researchers and machine learning engineers have tried to solve is detecting cancer in cross-sectioned microscopic slides for various tissue types. These slide images are called "whole slide images" (WSI) they are *huge* images, usually 1-2 GBs large and are digitized with fancy scanner machines.

Recently the FDA gave the first ever approval for a machine learning based digital pathology product [read more here](https://www.paige.ai/news/news-press-release1/). So the digital pathology industry is still growing and in need of great minds!

### What you should already
* Know the Python programming language and various packages like numpy and matplotlib
* Be familiar with PyTorch
* Have a basic understanding of machine and deep learning concepts

### You will learn
* To build a dense prediction model for image segmentation problems
* To understand how to convert research papers into usable networks using PyTorch

### Data set
The data is given as a set of 1024×1024 PNG images. These images are "tiles" or small squares of the original WSI because working with one gigabit file wouldn't fit on most machines and would be extremely slow for training.
Each input image (in the ```inputs``` directory) is an RGB image of a section of tissue,
and there a file with the same name (in the ```outputs``` directory) 
that has a dense labeling of whether or not a section of tissue is cancerous
(white pixels mean “cancerous”, while black pixels mean “not cancerous”).

The data has been pre-split for into test and training splits.
Filenames also reflect whether or not the image has any cancer at all 
(files starting with ```pos_``` have some cancerous pixels, while files 
starting with ```neg_``` have no cancer anywhere).
All of the data is hand-labeled, so the dataset is not very large.
This means that overfitting is a real possibility.

An example image, and its corresponding ground truth labeling, is shown below.
(And is contained in the downloadable dataset below).

![](http://liftothers.org/dokuwiki/lib/exe/fetch.php?w=200&tok=a8ac31&media=cs501r_f2016:pos_test_000072_output.png)
<img src="http://liftothers.org/dokuwiki/lib/exe/fetch.php?media=cs501r_f2016:pos_test_000072.png" width="200">

### Articles & Papers to read beforehand
* [U-Net: Convolutional Networks for Biomedical Image Segmentation | Arvix paper](https://arxiv.org/pdf/1505.04597.pdf)
* [Up Sampling Images | Toward Data Science](https://towardsdatascience.com/up-sampling-with-transposed-convolution-9ae4f2df52d0)

If you don't understand or are new to CNN's then chekcout this article as well
* [Intro to Convolutional Neural Networks | Towards Data Science](https://towardsdatascience.com/an-introduction-to-convolutional-neural-networks-eb0b60b58fd7)

___

## Part 1 - Implement a dense image segmentation network

#### 1.1 Dissecting the network topology

Below is an image from the U-Net paper, it is an illustration of what a U-Net "looks" like. You can probably guess why they named it "U" net.

![(Figure 1)](https://lh3.googleusercontent.com/qnHiB3B2KRxC3NjiSDtY08_DgDGTDsHcO6PP53oNRuct-p2QXCR-gyLkDveO850F2tTAhIOPC5Ha06NP9xq1JPsVAHlQ5UXA5V-9zkUrJHGhP_MNHFoRGnjBz1vn1p8P2rMWhlAb6HQ=w2400)

Let's take a look at all the parts of this network illustration.

We have, in order from the top-left of the "U", down to the bottom, and then up to the top-right of the "U":
1) 3x3 convoultion followed by a ReLU
2) 2x2 max pool
3) 2x2 up convolution
4) 1x1 final convolution
5) "Copy and crop" for each layer

TODO: dissect each part
Let's dissect each part; its importance and what PyTorch module we will use to implement it

TODO discuss layers

#### Food for thought

The simplest network you could implement (with all the desired properties)
is just a single convolution layer with two filters and no relu! 
Why is that? (of course it wouldn't work very well!)


#### 1.2 Creating the U-Net network

Below are the imports for all the packages and modules we will use. Run it before you run any other code.

In [1]:
!pip3 install torch
!pip3 install torchvision
!pip3 install tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
from torchvision import transforms, utils, datasets
from tqdm import tqdm
from torch.nn.parameter import Parameter
import pdb
import torchvision
import os
import gzip
import tarfile
import gc
from IPython.core.ultratb import AutoFormattedTB
__ITB__ = AutoFormattedTB(mode = 'Verbose',color_scheme='LightBg', tb_offset = 1)

assert torch.cuda.is_available(), "You need to request a GPU from Runtime > Change Runtime"

In [ ]:
class UNet(nn.module):
    def __init__(self, in_channels, out_channels, filter_start_depth=64, u_depth=4):
        super(UNet).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.depth = u_depth
        size = filter_start_depth

        self.conv = nn.Conv2d
        self.activation_func = nn.ReLU

        self.down_convs = nn.ModuleList()
        self.down_samples = nn.ModuleList()
        self.up_samples = nn.ModuleList()
        self.up_convs = nn.ModuleList()

        previous_size = self.in_channels
        current_size = size
        # down the U
        for i in range(self.depth) :
            self.down_convs.append(nn.Sequential(
                self.conv(previous_size, current_size, kernel_size=3, stride=1, padding=1),
                self.activation_func(),
                self.conv(current_size, current_size, kernel_size=3, stride=1, padding=1),
                self.activation_func(),
            ))
            self.down_samples.append(nn.MaxPool2d(kernel_size=2))
            previous_size = current_size
            current_size *= 2

        # bottom convolutions
        self.bottom = nn.Sequential(
            self.conv(previous_size, current_size, kernel_size=3, stride=1, padding=1),
            self.activation_func(),
            self.conv(current_size, current_size, kernel_size=3, stride=1, padding=1),
            self.activation_func()
        )

        # up the U
        for i in range(self.depth) :
            next_size = current_size//2
            self.up_samples.append(nn.Sequential(
                nn.ConvTranspose2d(current_size, next_size, kernel_size=2, stride=2, padding=0),
                self.activation_func(),
            ))
            self.up_convs.append(nn.Sequential(
                self.conv(next_size, next_size, kernel_size=3, stride=1, padding=1),
                self.activation_func(),
                self.conv(next_size, next_size, kernel_size=3, stride=1, padding=1),
                self.activation_func(),
            ))
            current_size = next_size

        # last convolutional layer
        self.final = self.conv(current_size, self.out_channels, kernel_size=1, stride=1, padding=0)

    def forward(self, x):
        # go down the U
        activations = []
        for i in range(self.depth):
            x = self.down_convs[i](x)
            activations.append(x)
            x = self.down_samples[i]()
        
        x = self.bottom(x)

        # back up the U
        for i in range(self.depth):
            x = self.up_samples[i](x)
            # figure out torch.cat
            x = self.up_convs[i](x)

        return self.final(x)

def pad_to_shape(tensor, out_shape):
    """
    Pads this image with zeroes to shp.
    Args:
        tensor: image tensor to pad
        shp: desired output shape
    Returns:
        Zero-padded tensor of shape shp.
    """
    if len(out_shape) == 4:
        pad = (0, out_shape[3] - tensor.shape[3], 0, out_shape[2] - tensor.shape[2])
    elif len(out_shape) == 5:
        pad = (0, out_shape[4] - tensor.shape[4], 0, out_shape[3] - tensor.shape[3], 0, out_shape[2] - tensor.shape[2])
    return F.pad(tensor, pad)

Next, we wrap our image dataset into a PyTorch dataset object to make it easier to load the data into memory and create batches.

In [ ]:
class CancerDataset(Dataset):
  def __init__(self, root, download=True, size=512, train=True):
    if download and not os.path.exists(os.path.join(root, 'cancer_data')):
      datasets.utils.download_url('http://liftothers.org/cancer_data.tar.gz', root, 'cancer_data.tar.gz', None)
      self.extract_gzip(os.path.join(root, 'cancer_data.tar.gz'))
      self.extract_tar(os.path.join(root, 'cancer_data.tar'))
    
    postfix = 'train' if train else 'test'
    root = os.path.join(root, 'cancer_data', 'cancer_data')
    self.dataset_folder = torchvision.datasets.ImageFolder(os.path.join(root, 'inputs_' + postfix) ,transform = transforms.Compose([transforms.Resize(size),transforms.ToTensor()]))
    self.label_folder = torchvision.datasets.ImageFolder(os.path.join(root, 'outputs_' + postfix) ,transform = transforms.Compose([transforms.Resize(size),transforms.ToTensor()]))

  @staticmethod
  def extract_gzip(gzip_path, remove_finished=False):
    print('Extracting {}'.format(gzip_path))
    with open(gzip_path.replace('.gz', ''), 'wb') as out_f, gzip.GzipFile(gzip_path) as zip_f:
      out_f.write(zip_f.read())
    if remove_finished:
      os.unlink(gzip_path)
  
  @staticmethod
  def extract_tar(tar_path):
    print('Untarring {}'.format(tar_path))
    z = tarfile.TarFile(tar_path)
    z.extractall(tar_path.replace('.tar', ''))

  def __getitem__(self,index):
    img = self.dataset_folder[index]
    label = self.label_folder[index]
    return img[0],label[0][0]
  
  def __len__(self):
    return len(self.dataset_folder)

In [ ]:
class SegmentationTrainer:
    def __init__(self):
        self.network = None
        self.optimizer = None
        self.loss_func = None
        self.train_dataloader = None 
        self.val_dataloader = None

        self.device = 'cuda'
        self.epoch_count = 0
        self.loop = None

    def increment_epoch_count(self):
        self.epoch_count += 1

    def train(self, num_epochs):
        train_losses = []
        val_losses = []
        train_accuracies = []
        val_accuracies = []

        for i in range(1, num_epochs+1):
            self.loop = tqdm(total=len(self.train_dataloader), position=0, leave=False)
            # Train
            print(f'\nTraining for epoch {i} of {num_epochs}')
            train_loss, train_acc = self.train_epoch_(self.train_dataloader, training=True)
            self.loop.close()

            self.loop = tqdm(total=len(self.val_dataloader), position=0, leave=False)
            # Validate
            print(f"\nValidation for epoch {i} of {num_epochs}")
            val_loss, val_acc = self.train_epoch_(self.val_dataloader, training=False)
            self.loop.close()
            

            train_losses.append(train_loss)
            val_losses.append(val_loss)
            train_accuracies.append(train_acc)
            val_accuracies.append(val_acc)

        return  train_losses, val_losses, train_accuracies, val_accuracies

    def train_epoch_(self, data_loader, training=True):
        if training:
            self.network.train() 
            torch.set_grad_enabled(True)
        else:
            self.network.eval()
            torch.set_grad_enabled(False)

        loss_sum = 0
        acc_sum = 0
        for imgs, labels in data_loader:
            imgs, labels = imgs.to(device, non_blocking=True), labels.to(device,non_blocking=True) # non_blocking is a speed up (async)
            
            if training:
                self.optimizer.zero_grad() # Set gradient to zero

            y_hat = self.network(imgs)
            loss = self.loss_func(y_hat, labels.long())
            loss_sum += loss.detach().sum().item()

            
            probs = y_hat.argmax(1)
            accuracy = (probs == labels).float().mean()
            acc_sum += accuracy.detach().item()

            mem_allocated = torch.cuda.memory_allocated(0) / 1e9

            self.loop.set_description('loss: {:.4f}, accuracy: {:.4f}, mem: {:.2f}'.format(loss.detach().sum().item(), accuracy, mem_allocated))
            self.loop.update(1)

            if training:
                loss.backward() # Compute gradient, for weight with respect to loss
                self.optimizer.step() # Take step in the direction of the negative gradient

        return loss_sum / len(data_loader.dataset), acc_sum / len(data_loader.dataset)

Now we instantiate anything we need for training. Datasets, the network, loss functions, etc.

In [ ]:
# Instantiate data sets
train_dataset = CancerDataset('/tmp', train=True)
val_dataset = CancerDataset('/tmp', train=False)

# Instantiate data loaders
train_loader = DataLoader(train_dataset,
                          shuffle=True,
                          batch_size=4,
                          num_workers=4,
                          pin_memory=True,)

val_loader = DataLoader(val_dataset,
                        shuffle=True,
                        batch_size=4,
                        num_workers=4,
                        pin_memory=True)

In [ ]:
# Instantiate the network
device = "cuda"

net = UNet(
    3,
    2,
    filter_start_depth=64,
    u_depth=4
).to(device)

# Instantiate loss function and optimizer
learning_rate = 1e-4
optimizer = torch.optim.Adam(
    net.parameters(),
    lr = learning_rate
)
loss_func = nn.CrossEntropyLoss().to(device)

Next we will create a training loop

In [ ]:
# This is what was talked about in the video for memory management

def scope():
  try:
    #your code for calling dataset and dataloader
    gc.collect()
    print(torch.cuda.memory_allocated() / 1e9)
    
    #for epochs:
    # Call your model, figure out loss and accuracy
    
  except:
    __ITB__()
    
scope()


WARNING: You may run into an error that says "RuntimeError: CUDA out of memory."

In this case, the memory required for your batch is larger than what the GPU is capable of. You can solve this problem by adjusting the image size or the batch size and then restarting the runtime. 


___

## Part 2 - Display performance

Plot performance over time

Please generate two plots:

 One that shows loss on the training and validation set as a function of training time. 

 One that shows accuracy on the training and validation set as a function of training time. 

 Make sure your axes are labeled!



In [ ]:
# Your plotting code here


**NOTE:**

Guessing that the pixel is not cancerous every single time will give you an accuracy of ~ 85%.
Your trained network should be able to do better than that (but you will not be graded on accuracy).
This is the result I got after 1 hour or training.

![](http://liftothers.org/dokuwiki/lib/exe/fetch.php?w=400&tok=d23e0b&media=cs501r_f2016:training_accuracy.png)
![](http://liftothers.org/dokuwiki/lib/exe/fetch.php?w=400&tok=bb8e3c&media=cs501r_f2016:training_loss.png)

___

## Part 3 - Generating predictions

This is the real test to see how well your model accomplished its task. We need it to work outside of a training environment and in the real world with novel data points.

Generate at least 5 predictions on the pos_test_000072.png image and display them as images. These predictions should be made at a reasonable interval (e.g. every epoch). 

You can load this image from the file pos_test_000072.png, or you can get it from the dataset object. It is item 172 of the validation dataset.
You can print both the data instance (x) and the ground-truth label (y_hat) to see how well your network predicts on that instance.

To do this, calculate the output of your trained network on the pos_test_000072.png image,
then make a hard decision (cancerous/not-cancerous) for each pixel.
The resulting image should be black-and-white, where white pixels represent areas
the network considers cancerous.

In [ ]:
# Code for testing prediction on an image
